## Ongoing Notes:
Key Problems:
1. Which dataset or datasets even contain the information we want
2. How do we determine when measurements were taken in the WQP dataset
3. How do we determine which param_codes to use (there are ~10,000!) via NWIS
4. How do we match streamflow gauges to their closest water quality gauge if one exists (lat/long data is available)
5. How do we work with the extremely reduced number of sites that have these niche water quality metrics<br>
    -Seems very unlikely that most if any sites will have all the ones listed in the proposal in one place

...in other words how do we spatially and temporarily match streamflow data with water quality data
<br><br>

TODO:
1. Determine relevant gauge counts for at least some of these metrics, per state
2. Determine type of data returned, time-series, or otherwise
<br><br>

Comments:<br>
How granular will the water quality data be ultimately? Is building a robust water quality profile for a few gauges in locations where EAR is viable the focus, or is having a general idea of an entire watershed regions water quality the focus?

## Initial Water Quality Data Exploration

In [1]:
#Python3.10
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import contextily as cx
from importlib import reload
from typing import IO
from IPython.display import display

from datetime import datetime, timedelta

# USGS Data retreival tool
from dataretrieval import nwis, utils, codes

# Custom modules are imported in multiple locations to faciliate easy reloading when edits are made to their respective files
import Src.classes as cl
import Src.func as fn
reload(cl)
reload(fn)

# TODO: Look into the warning that this is disabling. It doesn't appear to be significant for the purposes of this code but should be understood
pd.options.mode.chained_assignment = None

#pd.options.mode.chained_assignment = 'warn'

In [7]:
df = nwis.get_record(sites='11447650', service='qwdata', start='2019-10-01', end='2020-09-30')

C:\Users\alekh\AppData\Roaming\Python\Python310\site-packages\dataretrieval\nwis.py:180: UserWarning: WARNING: Starting in March 2024, the NWIS qw data endpoint is retiring and no longer receives updates. For more information, refer to https://waterdata.usgs.gov.nwis/qwdata and https://doi-usgs.github.io/dataRetrieval/articles/Status.html or email CompTools@usgs.gov.
  warnings.warn(('WARNING: Starting in March 2024, the NWIS qw data endpoint is '


In [13]:
print(df.columns.to_list())
print(len(df.columns.to_list()))
print(df['p00300'].count())

['agency_cd', 'site_no', 'sample_dt', 'sample_tm', 'sample_end_dt', 'sample_end_tm', 'sample_start_time_datum_cd', 'tm_datum_rlbty_cd', 'coll_ent_cd', 'medium_cd', 'project_cd', 'aqfr_cd', 'tu_id', 'body_part_id', 'hyd_cond_cd', 'samp_type_cd', 'hyd_event_cd', 'sample_lab_cm_txt', 'p00003', 'p00004', 'p00009', 'p00010', 'p00020', 'p00025', 'p00032', 'p00035', 'p00041', 'p00061', 'p00063', 'p00065', 'p00095', 'p00191', 'p00300', 'p00301', 'p00400', 'p00403', 'p00405', 'p00530', 'p00600', 'p00602', 'p00605', 'p00607', 'p00608', 'p00613', 'p00618', 'p00623', 'p00625', 'p00631', 'p00660', 'p00665', 'p00666', 'p00671', 'p00681', 'p00688', 'p00689', 'p00691', 'p00694', 'p00900', 'p00904', 'p00915', 'p00925', 'p00930', 'p00931', 'p00932', 'p00935', 'p00940', 'p00945', 'p00950', 'p00955', 'p01000', 'p01020', 'p01046', 'p01080', 'p01085', 'p01130', 'p01145', 'p29801', 'p29802', 'p30207', 'p30209', 'p31795', 'p31797', 'p31863', 'p31872', 'p32295', 'p32296', 'p32297', 'p32298', 'p32299', 'p32300'